Color map for well distributed colors

In [12]:
#Colors to make every graph costant
colorMapWorld = {}


colorMapRegions = {}

colorMapProvinces = {}



def colorMap(states,wrp):
    for i in range(len(states)):
        color = list(np.random.random( size=3))
        color=color.append(1)
        if(wrp==0):
            colorMapWorld[states[i]]=color
            
        elif(wrp==1):
            colorMapRegions[states[i]]=color
        else:
            colorMapProvinces[states[i]]=color
    

        

Function to plot data about the growth starting from a specific number of cases. The countries for which the values are plotted are passed to the funcion as a list.

In [1]:
def plotGrowthDataFromFirstCases(dataset, countries_list, column_reference,
                               graph_title, starting_number, y_axis_max_limit = -1):
    """This procedure allow to plot data relative to the growth of
        a specific value starting from the first cases.
        Parameters:
        dataset -> is the dataset where the data are retreived
        countries_list -> is the list of countries which data must be plotted
        column_reference -> is the name of the column under analysis
        graph_title -> is the name of the graph that must be showed
        starting_number -> is the minimum number of cases from each the growth
            must be plotted
        y_axis_max_limit -> is the maximum value for the y axis plot, default value is -1,
            that means plot with the maximum range
    """
    for country in countries_list:
        country_growth_total_cases = dataset[dataset['Country'] == country]
        country_growth_total_cases = country_growth_total_cases[country_growth_total_cases[column_reference] >= starting_number]
        plt.plot(range(len(country_growth_total_cases)),country_growth_total_cases[column_reference],color=colorMapWorld[country], label = country)
    plt.xticks(rotation = 90)
    plt.legend()
    plt.title(graph_title)
    if (y_axis_max_limit > 0):
        plt.ylim(0, y_axis_max_limit)
    plt.show()

Define a function to evaluate the number of cases per million of inhabitants.

In [13]:
def evaluateRatePerNumberOfMillion(dataset, ref_column, new_column_name):
    """This funcion allows to add a new columns to the input dataset
       containing the number of cases of the reference column out of the
       population.
       Parameters:
       dataset -> is the input dataset containing all the necessary information.
           It is import it has a column "Population" with the total number of
           inhabitants.
       ref_column -> is the column under evaluation
       new_column_name -> is the name of the column where the value is stored
    """
    
    dataset[new_column_name] = dataset[ref_column]/dataset['Population'] * 1000000
    return dataset

Define a function that create a new dataframe containing the growth rate for each country. It is possible to specify the time interval to consider, expressed in days.

In [15]:
def evaluateGrowthRate(dataset, number_of_days, reference_column):
    """This function allows to evaluate the growth rate of the reference
       column in input, gathering the data from the dataset in input with
       a specific delay expressed in days. The starting and ending date
       considered is based by the minumum and maximum date present in the
       input dataset. The dataframe in input must have a column 'Date'
       containing the list of reference date. The value returned is a
       new dataframe containing for each country a specific column and the
       value in each row is the percentage growth calculated from the previous 
       date.
       Parameters:
       dataset -> is the dataset to gather the data
       number_of_days -> is the frequency in which the evaluation is done
       reference_column -> is the column in the dataset in input to consider
           for evaluating the growth
    """
    #get the minimum date to consider
    minimum_date = dataset['Date'].min()
    #get the maximum date to consider
    maximum_date = dataset['Date'].max()
    #create the list of country to consider
    list_of_country = dataset['Country'].unique()
    #Create a new dataframe with the list of country in each column
    #and the list of date as index
    date_list = pd.date_range(start = minimum_date, end = maximum_date, 
                              freq = str(number_of_days) + 'D')
    #create a new dataframe with country as columns and date as index
    #filled with 0 values
    zero_data = np.zeros(shape=(len(date_list),len(list_of_country)))
    df_growth = pd.DataFrame(zero_data, index = date_list, columns = list_of_country)
    #iterate on all the index date
    for index_date in range(len(date_list)):
        #first row is not considered because there is not a previus value
        #for which the growth can be evaluated
        if index_date == 0:
            continue
        else:
            #get the string date that has to be evaluated
            current_date = str(date_list.values[index_date])[:10]
            #get the previous date
            previous_date = str(date_list.values[index_date - 1])[:10]
            #iterate on all the countries present in the input dataset
            for country in list_of_country:
                #get the list of value available for the country in exam
                country_values = dataset.loc[dataset['Country'] == country]
                #get the row value for the date/country in exame
                current_value = country_values.loc[dataset['Date'] == current_date]
                #get the previous row value for the date/country in exame
                previous_value = country_values.loc[dataset['Date'] == previous_date]
                #verify that the two values are not empty, otherwise skip to the next
                #iteration
                #get all death
                total_value=country_values[reference_column].iloc[-1]
                #total_value=country_values.sum[reference_column]
                if len(current_value) > 0 and len(previous_value) > 0:
                    #if values are not empty, extract the two number used to evaluate
                    #the growth
                    current_value = current_value[reference_column].values[0]
                    previous_value = previous_value[reference_column].values[0]
                    
                    #verifies that the previous value is not 0 to avoid division error
                    if previous_value > 0:
                        #evaluate the percentage growth between the 2 consecutive values
                        growth_value = (current_value - previous_value) / total_value * 100
                        #set the evaluated value in the dataframe, at the right location
                        df_growth.loc[current_date,country] = growth_value
    #return the new dataframe with the outcomes
    return df_growth

Define a function to create a matrix of value

In [5]:
def get_matrix_dataset(dataset, column_to_consider):
    """
    This function produce a new dataframe where all the columns
    represent a country, each row represent a date and the value
    of the matrix is the value specified in the input 'column_to_consider'
    Parameters:
    dataset -> is the dataset organized in rows
    column_to_consider -> is the value contained in the matrix
    """
    #get the minimum date to consider
    minimum_date = dataset['Date'].min()
    #get the maximum date to consider
    maximum_date = dataset['Date'].max()
    #create the list of country to consider
    country_list = dataset['Country'].unique()
    #set Date column as index
    dataset.set_index('Date',inplace = True)
    #create a new dataframe with the list of country in each column
    #and the list of date as index
    date_list = pd.date_range(start = minimum_date, end = maximum_date, 
                              freq = '1' + 'D')
    zero_data = np.zeros(dtype=int, shape=(len(date_list),len(country_list)))
    df = pd.DataFrame(zero_data, index = date_list, columns = country_list)
    for date in date_list:
        date = str(date)[:10]
        for country in country_list:
            date_rows = dataset.loc[dataset['Country'] == country]
            try:
                value = date_rows.loc[date, column_to_consider]
                df.loc[date,country] = value
            except:
                pass
    return df

Define a function to evaluate the moving average.

In [2]:
def evaluate_moving_average(dataset, period, time_series = True):
    '''
    This function take a time series matrix dataset in input,  
    if not it must be present a column 'Date', and a column for each
    country. Create a new dataset with the same column but with the
    value of each cell that is the average of the previous n value
    where n is the period in input. The first n rows of the original
    dataset are truncated in the final dataset.
    Parameters:
    dataset -> input dataset where the moving average is computed.
    period -> is the number of sample to evaluate the moving average.
    time_serie -> specify if the dataset in input is a time series.
    '''
    #reset data index if necessary
    if time_series:
        dataset.reset_index(inplace = True)
        dataset.rename(columns = {'index':'Date'}, inplace = True)
    
    #create a new empty dataframe with a number of rows less of the
    #period of time input but with the same country column
    date_list = dataset['Date'][period:]
    #set Date column as index
    dataset.set_index('Date',inplace = True)
    country_list = dataset.columns
    #create 0 value matrix to insert into the new dataframe
    zero_data = np.zeros(dtype=int, shape=(len(date_list),len(country_list)))
    df = pd.DataFrame(zero_data, index = date_list, columns = country_list)
    
    #evaluate the moving average of the input dataset and 
    #insert the value in the new dataset.
    #Iterate on each date
    for date_index in range(len(date_list)):
        #Iterate on each country
        for country in country_list:
            #Evaluate the average of the previous n-1 cell and the 
            #current cell for the country in input
            average = dataset.iloc[date_index-period:date_index][country].mean()
            #store the average in the new dataset
            df.iloc[date_index][country] = average
    return df

Plot for the lockdown moving average

In [5]:
'''
Inputs:
Checks if it's the peak of the curve, or it hasn't reach it yet
column-> it's the dataset column we are analyzing. Ex: Afghanistan,Italy ecc...
peakIndex-> It's the supposed peak that we have to check

Output->
It returns 0 if its not the peak yet, otherwise it returns the peak as it is
'''
def peakChecker(column,peakIndex):
    if(peakIndex+datetime.timedelta(days=1) in column.index): #checking if there is a day after the peak
        return peakIndex
                    
                    
    else:
        return 0



      
'''
10 graphs, one for each country in the top ten with a lockdown line and a counter

Inputs:
top_ten_moving_average-> the top ten of the countries with the moving mean
df_lockdown_states->dataset with the lockdown information
df_moving_average_-> dataset with the moving average of every country
name-> name of the Y axis of the graph


Output->
Prints the 10 graphs
'''    
def plotWithLockdown(top_ten_moving_average,df_lockdown_states,df_moving_average_,name):
    i=0 #Subplot indexer, can be from 0 to 4 and indexes the rows
    j=0 #Subplot indexer, can be from 0 to 1 and indexes the columns
    fig, axs = plt.subplots(5, 2,figsize=(15,20)) #Subplot 5x2 
    fig.tight_layout(pad=7.0) #to distanciate better the graphs
    
    for d in range(0,len(top_ten_moving_average)):
        #Calculating the peak of the curve, taking the max value for a column in the top ten
        lockdown_date=df_lockdown_states['Beginning Date'][df_lockdown_states['Country']==top_ten_moving_average[d]]
        peak=df_moving_average_[top_ten_moving_average[d]].max()
        peak=df_moving_average_[top_ten_moving_average[d]][df_moving_average_[top_ten_moving_average[d]]==peak]
        
        #then checking the peak to see if it's the last value (counter=Peak not reached) or not (counter = lenght between lockdown and peak) 
        newPeak=peakChecker(df_moving_average_[top_ten_moving_average[d]],peak.index[0])
        if(newPeak!=0):
            day_counter=len(df_moving_average_[top_ten_moving_average[d]][lockdown_date.iloc[0]:peak.index[0]])
        else:
            day_counter='Peak not reached yet'

        #building the subplots
        axs[i, j].plot( df_moving_average_.index,df_moving_average_[top_ten_moving_average[d]],color=colorMapWorld[top_ten_moving_average[d]], label= '')
        axs[i, j].set_title(top_ten_moving_average[d])
        axs[i, j].set(xlabel='Dates', ylabel=name)
        axs[i, j].tick_params(labelrotation=45)
        axs[i, j].annotate('Counter from the lockdown: '+str(day_counter), xy=(10, 170), xycoords='axes points',
                size=10, ha='left', va='top',
                )
        try:
            axs[i, j].axvline(x=lockdown_date.iloc[0],color='red')
        except:
            print()
       
        #Index incrementing. It must be done manually and not with a for otherwise it's impossibile
        j=j+1
    
        if(j==2):
            j=0
            i=i+1
            if(i==5):
                i=0
'''
Output->List containing days passed between lockdown and the peak for each nations, and not just the top ten

Input:
countires-> list of all countries
df_lockdown_states->dataset with the lockdown information
df-> dataset with the cases of every country


''' 

def peakDays(countries,df_lockdown_states,df):
    days=[]
    
    
    for d in range(0,len(countries)):
        #Calculating the peak of the curve, taking the max value for a column in the top ten
        lockdown_date=df_lockdown_states['Beginning Date'][df_lockdown_states['Country']==countries[d]]
        peak=df[countries[d]].max()
        peak=df[countries[d]][df[countries[d]]==peak]
        
        #then checking the peak to see if it's the last value (counter=Peak not reached) or not (counter = lenght between lockdown and peak) 
        newPeak=peakChecker(df[countries[d]],peak.index[0])
        if(newPeak!=0):
            day_counter=len(df[countries[d]][lockdown_date.iloc[0]:peak.index[0]])
        else:
            day_counter=0
        days.append(day_counter)
    return days

Plot creator:
This is for the plot for italy's data    

In [3]:

'''
Input:
x-> the range of days passed since the thing we want to represent Ex: if a month passed, x=range(1,30)
y-> is an array of datasets row. Each component of the list is a row containing a province/region with 
the values for the matter in exam following. Ex: if we are analyzing the deaths, every component of the list will be: 
'Abruzzo,0,20,...,2000' The rows goes on as the same amount of days passed
title-> The name given to the graph. It's a list of two elements, cause and effect of the graph. Ex: ['Number of deaths','Region']

Output: It prints the graph
'''
def condition_plot(x,y,title):
    plt.figure()
    
    for i in range(0,len(y)):
        if(title[1]=='regions'):
            plt.plot(x,y[i][1:],color=colorMapRegions[y[i][0]],label=y[i][0])
        else:
            plt.plot(x,y[i][1:],color=colorMapProvinces[y[i][0]],label=y[i][0])
    plt.legend(bbox_to_anchor=(1, 1.05))


    plt.xlabel('Days')
    plt.ylabel(title[0])
    plt.title(title[0]+' for '+title[1])

    plt.show()
    plt.clf()




Top ten regions/provinces: this is to have a top ten list of regions/provinces

In [ ]:
'''
Input:
dataset-> it's the dataset in which we have our data to classify
name_column-> it's the name/index of the column where the names to assign to each value  is stored. 
If Abruzzo is the best region for example, we want to use that name, and that name is stored at column '0'

Output:
A list containing dataset rows, in which we have the region and the values that will be used in the graphs
'''

def top_ten_list(dataset,name_column):
    #loading the list
    y=[]  #list of the condition for provinces/regions, in which every entry of the list is a day (its a list of lists)
    for i in range(dataset.shape[0]):
        y.append(dataset.iloc[i,name_column:]) #position 1 is the province name, which we need for the label in the graph

    #Let's start with the top ten 
    top_ten=[]
    last_day=[]
    #first let's determine the last day how many people has a certain condition
    for i in range(name_column,len(y)):
        last_day.append(y[i][-1])
    last_day.sort(reverse = True)

  
    #then let's have only the first 10
    for i in range(name_column,len(y)):
        if y[i][-1] in last_day[0:10]:
            top_ten.append(y[i])
    
    return top_ten

Moving mean italy

In [3]:
'''
Input:
data-> Dataset
window_size-> The size of the moving mean

Output:
An array with moving averages for each rows of the dataset
'''

def moving_mean(data,window_size):
    i=0
    
    moving_averages=[]
    while i < len(data) - window_size + 1:
        this_window = data[i : i + window_size]
        #print(this_window[0][1:])
        window_average = sum(this_window[1:]) / window_size
        moving_averages.append(window_average)
        i += 1

    return moving_averages

'''
Input:
top_ten-> list of datasets rows, containg name of the region/provinces and its values
window_size-> The size of the moving mean

Output:
An array with moving averages for each rows of the dataset with the name of the region/province
'''

def moving_mean_with_names(top_ten,windows_size):

    moving_averages=[]
    for i in range(len(top_ten)):
        moving_averages.append(moving_mean(top_ten[i],windows_size))

 

    for i in range(len(top_ten)):
        moving_averages[i][0]=top_ten[i][0]
    return moving_averages

Autocorrelation

In [12]:
'''
10 graphs, one for each country in the top ten for the autocorrelation

Inputs:
top_ten_moving_average-> the top ten of the countries with the moving mean

df_moving_average_-> dataset with the moving average of every country
total-> boolean for total autocorrelation (True) or partial (False)
lag-> lag of the autocorrelation plot

Output->
Prints the 10 graphs
'''    
def autocorrelationPlot(name,values,lag):
    
    fig, axs = plt.subplots(1, 2,figsize=(15,5)) #Subplot 5x2 
    fig.tight_layout(pad=7.0) #to distanciate better the graphs
    values=datasetAlignment(values[name])
    
    
    try:        
        plot_acf(values,color=colorMapWorld[name], label= '',lags=lag,ax=axs[0],title=name+" autocorrelation")
          
        plot_pacf(values,color=colorMapWorld[name], label= '',lags=lag,ax=axs[1],title=name+" partial autocorrelation")
    except:
        
        plot_acf(values,color=colorMapRegions[name], label= '',lags=lag,ax=axs[0],title=name+" autocorrelation")
       
        plot_pacf(values,color=colorMapRegions[name], label= '',lags=lag,ax=axs[1],title=name+" partial autocorrelation")
       
              
        

'''

Decomposition of the autocorrelation plot
'''
def decompositionPlot(name,values):
  
    
    
        values=datasetAlignment(values[name])
        
        rcParams['figure.figsize'] = 18, 8
        #building the subplots
        decomposition=seasonal_decompose(values, model='additive', period=int((len(values)-1)/2),extrapolate_trend='freq')
      
        fig, axes = plt.subplots(4, 1, sharex=True)
        try:
        
            decomposition.observed.plot(ax=axes[0], legend=False, color=colorMapWorld[name])
            axes[0].set_ylabel('Observed')
            decomposition.trend.plot(ax=axes[1], legend=False, color=colorMapWorld[name])
            axes[1].set_ylabel('Trend')
            decomposition.seasonal.plot(ax=axes[2], legend=False, color=colorMapWorld[name])
            axes[2].set_ylabel('Seasonal')
            decomposition.resid.plot(ax=axes[3], legend=False, color=colorMapWorld[name])
            axes[3].set_ylabel('Residual')
        except:
            decomposition.observed.plot(ax=axes[0], legend=False, color=colorMapRegions[name])
            axes[0].set_ylabel('Observed')
            decomposition.trend.plot(ax=axes[1], legend=False, color=colorMapRegions[name])
            axes[1].set_ylabel('Trend')
            decomposition.seasonal.plot(ax=axes[2], legend=False, color=colorMapRegions[name])
            axes[2].set_ylabel('Seasonal')
            decomposition.resid.plot(ax=axes[3], legend=False, color=colorMapRegions[name])
            axes[3].set_ylabel('Residual')
        
       
        
            
        
def datasetAlignment(matrix):
    for i in range(0,len(matrix)):
        if(matrix[i]>0):
            return matrix[i:]
        
    

In [5]:
'''
Linear regression cluster operations

Input:
x-> indipendent variable
y-> dipendent variable
labelx and y-> the name of the plot labels

output:
plot of the regression
accuracy informations (R^2 and mean square error)
'''


def linearRegression(x,y,labelx,labely):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    regr = LinearRegression().fit(x_train.to_numpy().reshape(-1, 1),y_train)


    # Make predictions using the testing set
    popDens_y_pred = regr.predict(x_test.to_numpy().reshape(-1, 1))


    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print('Mean squared error: %.2f'
          % mean_squared_error(y_test, popDens_y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'
          % r2_score(y_test, popDens_y_pred))

    # Plot outputs
    plt.scatter(x_test.to_numpy().reshape(-1, 1),y_test,  color='red')
    plt.plot(x_test.to_numpy().reshape(-1, 1), popDens_y_pred, color='blue', linewidth=3)
    plt.xlabel(labelx)
    plt.ylabel(labely)
    plt.show()

In [7]:
'''
Counter of deaths until a day X

Input:
df_matrix_new_cases-> matrix with the number of deaths each day
df_regression_friendly-> matrix with the data about days and countries
days-> how many days ago from the lockdown

Output->number of deaths until day X

'''


def numberOfDeathsDayX(df_matrix_new_cases,df_regression_friendly,day):
    i=0
    numberOfDeaths=[]
    for column in df_regression_friendly['Country']:

        row=df_matrix_new_cases[df_matrix_new_cases[column].index==pd.to_datetime(df_regression_friendly['Beginning Date'].iloc[i])-datetime.timedelta(days=day)]
     
        dateToStop=df_matrix_new_cases[:row.index[0]]
        
      
        numberOfDeaths.append(sum(dateToStop[column]))
        i=i+1
    return numberOfDeaths

In [3]:
def scatterCases(top_ten_new_cases,df_matrix_new_cases,threshold,InOneGraph):
    plt.figure(figsize=(15,7.5))

    for country in top_ten_new_cases:
        for i in range(0,len(df_matrix_new_cases[country])):
            if(df_matrix_new_cases[country][i]>threshold):
                 matrix=df_matrix_new_cases[country][i:]
                 break
        x=df_regression_friendly[df_regression_friendly['Country']==country]  
        try:
            numberOfDaysThreshold=((x['Beginning Date']- matrix.index[0]).iloc[0]).days
        except:
            numberOfDaysThreshold='n'
        if(numberOfDaysThreshold=='n'): 
            clr='black'
        elif(numberOfDaysThreshold<0):
            clr='green'
        elif(numberOfDaysThreshold>=0 and numberOfDaysThreshold<10):
            clr='red'
        else:
            clr='blue'
        if(InOneGraph):
            plt.scatter(matrix.index,matrix,color=clr)
        else:
            plt.scatter(matrix.index,matrix,color=clr)
            plt.title(country)
            plt.show()
            plt.clf()
            plt.figure(figsize=(15,7.5))


In [3]:
'''
ARIMA
'''


def evaluate_arima_model(X, arima_order,trainDim):
    # prepare training dataset
    train_size = int(len(X) * trainDim)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    error = mean_squared_error(test, predictions)
    return error

def evaluate_sarimax_model(X, arima_order,trainDim):
    # prepare training dataset
    train_size = int(len(X) * trainDim)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = SARIMAX(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    error = mean_squared_error(test, predictions)
    return error
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values,trainDim,arima):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                if arima:
                    try:
                        mse = evaluate_arima_model(dataset, order,trainDim)
                        if mse < best_score:
                            best_score, best_cfg = mse, order
                        
                    except:
                        continue
                else:
                    try:
                        mse = evaluate_sarimax_model(dataset, order,trainDim)
                        if mse < best_score:
                            best_score, best_cfg = mse, order
                       
                    except:
                        continue
                    
   
    return best_cfg

# load dataset
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')



def autoArima(dataset,trainDim,stationary):
    p_values = [4,5,6]
    d_values = [0,1,2]
    q_values = [2,3,4,5,6]
    if (not stationary):
        best_cfg=evaluate_models(dataset.values, p_values, d_values, q_values,trainDim,True)
        print('ARIMA: ')
    else:
        best_cfg=evaluate_models(dataset.values, p_values, d_values, q_values,trainDim,False)
        
        print('SARIMAX: ')
        
    
        
        
        
        
    print(best_cfg)
    X = dataset.values
    size = int(len(X) * trainDim)
    train, test = X[0:size], X[size:len(X)]
    history = [x for x in train]
    predictions = list()
    for t in range(len(test)):
        if stationary:
            model = SARIMAX(history, order=best_cfg)
        else:
            model = ARIMA(history, order=best_cfg)
    
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
    
        obs = test[t]
        history.append(obs)
        print('predicted=%f, expected=%f' % (yhat, obs))
    error = mean_squared_error(test, predictions)

    print('Test MSE: %.3f' % error)
    # plot
    plt.plot(test)
    plt.plot(predictions, color='red')
    plt.show()

# Miscellanous

This code is when you downlaod the csv files on you pcs file by file.
It's no use for this notebook, avoid to compile it, it will not work.
There is also scrap code, here in case i need it again

In [ ]:
'''
def autocorrelationPlot(top_ten_moving_average,df_moving_average_,total,lag):
    i=0 #Subplot indexer, can be from 0 to 4 and indexes the rows
    j=0 #Subplot indexer, can be from 0 to 1 and indexes the columns
    fig, axs = plt.subplots(5, 2,figsize=(15,20)) #Subplot 5x2 
    fig.tight_layout(pad=7.0) #to distanciate better the graphs
    
    for d in range(0,len(top_ten_moving_average)):
        

        try:#building the subplots
            if(total):
                plot_acf(df_moving_average_[top_ten_moving_average[d]],color=colorMapWorld[top_ten_moving_average[d]], label= '',lags=lag,ax=axs[i, j],title=top_ten_moving_average[d]+" autocorrelation")
            else:
                plot_pacf(df_moving_average_[top_ten_moving_average[d]],color=colorMapWorld[top_ten_moving_average[d]], label= '',lags=lag,ax=axs[i, j],title=top_ten_moving_average[d]+" partial autocorrelation")
        except:
            if(total):
                plot_acf(df_moving_average_[top_ten_moving_average[d]],color=colorMapRegions[top_ten_moving_average[d]], label= '',lags=lag,ax=axs[i, j],title=top_ten_moving_average[d]+" autocorrelation")
            else:
                plot_pacf(df_moving_average_[top_ten_moving_average[d]],color=colorMapRegions[top_ten_moving_average[d]], label= '',lags=lag,ax=axs[i, j],title=top_ten_moving_average[d]+" partial autocorrelation")
       
                
        
        j=j+1
    
        if(j==2):
            j=0
            i=i+1
            if(i==5):
                i=0
                
def decompositionPlot(top_ten_moving_average,df_moving_average_):
  
    
    
    for d in range(0,len(top_ten_moving_average)):
        
        rcParams['figure.figsize'] = 18, 8
        #building the subplots
        decomposition=seasonal_decompose(df_moving_average_[top_ten_moving_average[d]], model='additive', period=int((len(df_moving_average_)-1)/2))
        decomposition.plot()

        
        fig, axes = plt.subplots(4, 1, sharex=True)
        decomposition.observed.plot(ax=axes[0], legend=False, color=colorMapWorld[top_ten_moving_average[d]])
        axes[0].set_ylabel('Observed')
        decomposition.trend.plot(ax=axes[1], legend=False, color=colorMapWorld[top_ten_moving_average[d]])
        axes[1].set_ylabel('Trend')
        decomposition.seasonal.plot(ax=axes[2], legend=False, color=colorMapWorld[top_ten_moving_average[d]])
        axes[2].set_ylabel('Seasonal')
        decomposition.resid.plot(ax=axes[3], legend=False, color=colorMapWorld[top_ten_moving_average[d]])
        axes[3].set_ylabel('Residual')
        
        
'''
'''
print(len(population_time_line_dataset))
print(len(time_line_dataset))
difference =pd.Index(time_line_dataset['denominazione_provincia'].tolist()).symmetric_difference( population_time_line_dataset['denominazione_provincia'])
print(difference)
'''

'''

five_days_before=numberOfDeathsDayX(df_matrix_new_cases,df_regression_friendly,5)
data_lockdown=numberOfDeathsDayX(df_matrix_new_cases,df_regression_friendly,0)
growth_before_lockdown=(np.asarray(data_lockdown)-np.asarray(five_days_before))/np.asarray(data_lockdown)


for filename in listdir("./dati-province\\"):
   
    temp_data= pd.read_csv("./\\"+filename)

    time_line_dataset[filename[-12:-4]]=temp_data['totale_casi']
'''
'''
df_for_models=pd.DataFrame(df_regression_friendly['Country']).copy()


df_for_models['Growth before lockdown']=growth_before_lockdown
df_for_models['Growth before lockdown']=df_for_models['Growth before lockdown'].fillna(0)

df_for_models

df_for_models['Delta growth']=np.asarray(data_lockdown)-np.asarray(five_days_before)
df_for_models

df_for_models['Lockdown cases']=np.asarray(data_lockdown)
df_for_models['5 days before lockdown cases']=np.asarray(five_days_before)
df_for_models


df_for_models['Growth * Delta']=df_for_models['Growth before lockdown']*df_for_models['Delta growth']
df_for_models

df_for_models['Peak after lockdown']=peakDays(df_for_models['Country'],df_lockdown,df_matrix_new_cases)
df_for_models

linearRegression(df_for_models['Growth * Delta'],df_for_models['Peak after lockdown'] ,'Growth * delta','Peak after lockdown')

'''

Dickey-Fuller test

In [2]:
def dickeyFuller (series):
    adf = adfuller(series)
    print('ADF Statistic: {}'.format(adf[0]))
    print('p-value: {}'.format(adf[1]))
    print('Critical Values:')
    for key, value in adf[4].items():
        print('\t{}: {}'.format(key, value))
    return adf

In [ ]:
def Union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list 


In [7]:
def curveAfterPeak(dates,data):
    # date format is not suitable for modeling, let's transform the date into incrementals number starting from April 1st
  # April 1st is the 37th day of the series
    if(data.max()!=data.iloc[-1]):
        
        starting_date=23
        day_numbers = []
        plt.figure(figsize=(15,7.5))
        for i in range(1, len(dates[window-2:])):
            day_numbers.append([i])
        X = day_numbers
        # # let's train our model only with data after the peak
        X = X[starting_date:]
        y = data[starting_date:]
        # Instantiate Linear Regression
        linear_regr = linear_model.LinearRegression()
        # Train the model using the training sets
        linear_regr.fit(X, y)
        plt.scatter(X,y)
        print ("Linear Regression Model Score: %s" % (linear_regr.score(X, y)))
        return X,y,starting_date, linear_regr
    else:
        print('Peak not reached yet')

In [4]:
def predictTrain(X, Y, TrainX,TrainY,starting_date,listDate, linear_regr):
    # convert date of the epidemic peak into datetime format
    future_days = 100
    date=datetime.datetime.strftime(listDate[starting_date], '%Y-%m-%dT%H:%M:%S')
    date_zero = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S')
    # creating x_ticks for making the plot more appealing
    date_prev = []
    x_ticks = []
    step = 5
    data_curr = date_zero
    x_current = peak_date
    n = int(future_days / step)
    for i in range(0, n):
        date_prev.append(str(data_curr.day) + "/" + str(data_curr.month))
        x_ticks.append(x_current)
        data_curr = data_curr + timedelta(days=step)
        x_current = x_current + step
 
    
    plt.figure(figsize=(15,7.5))
    y_pred = linear_regr.predict(TrainX)
    error = max_error(TrainY, y_pred)

    X_test = []

    for i in range(starting_date, starting_date + future_days):
        X_test.append([i])
    y_pred_linear = linear_regr.predict(X_test)
    y_pred_max = []
    y_pred_min = []
    for i in range(0, len(y_pred_linear)):
        y_pred_max.append(y_pred_linear[i] + error)
        y_pred_min.append(y_pred_linear[i] - error)
    plt.grid()
    plt.scatter(X,Y)
    # plot linear regression prediction
    plt.plot(X_test, y_pred_linear, color='green', linewidth=2)
    # plot maximum error
    plt.plot(X_test, y_pred_max, color='red', linewidth=1, linestyle='dashed')
    #plot minimum error
    plt.plot(X_test, y_pred_min, color='red', linewidth=1, linestyle='dashed')
    plt.xlabel('Days')
    plt.xlim(starting_date, starting_date + future_days)
    plt.xticks(x_ticks, date_prev)


    plt.ylabel('new cases')
    plt.yscale("linear")
    plt.ylim(0, max(TrainY))
    plt.show()